In [5]:
!pip install nltk
!pip install googletrans==4.0.0-rc1
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mrx\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
import nltk
from nltk.corpus import wordnet
from googletrans import Translator
import cx_Oracle

translator = Translator()

# Paramètres de connexion à la base de données
dsn = cx_Oracle.makedsn("localhost", "1521", service_name="orcl")
conn = cx_Oracle.connect(user="sys", password="123", dsn=dsn, mode=cx_Oracle.SYSDBA)

try:
    cursor = conn.cursor()
    query = "SELECT RefSOUSSOUSDomaineF, MotsCles FROM SOUSSOUSDomaineFormation"
    cursor.execute(query)

    for row in cursor:
        ref = row[0]  
        mots_cles_lob = row[1]
        mots_cles = mots_cles_lob.read() if mots_cles_lob else ''
        mots_individuels = mots_cles.split(',')

        for mot in mots_individuels:
            mot = mot.strip()
            if mot:  # Vérifie si le mot est non vide
                try:
                    # Tentez la traduction
                    translation = translator.translate(mot, dest='en')
                except Exception as e:
                    # Gérez l'exception et continuez
                    print(f"Erreur de traduction pour le mot '{mot}': {e}")
                    continue  # Continue avec le prochain mot

                try:
                    insert_cursor = conn.cursor()
                    insert_query = "INSERT INTO TraductionMots (ID, MotOriginal, MotTraduit) VALUES (:id, :mot_orig, :mot_trad)"
                    insert_cursor.execute(insert_query, id=ref, mot_orig=mot, mot_trad=translation.text)
                    conn.commit()
                    insert_cursor.close()
                except cx_Oracle.DatabaseError as e:
                    # Gérez l'erreur de base de données et continuez
                    error, = e.args
                    print(f"Erreur de base de données: {error.code} - {error.message}")
                    continue  # Continue avec le prochain mot

finally:
    cursor.close()
    conn.close()


Erreur de traduction pour le mot 'responsable': 'Translator' object has no attribute 'raise_Exception'
Erreur de traduction pour le mot 'achats': 'Translator' object has no attribute 'raise_Exception'
Erreur de traduction pour le mot 'diplomantcpf': 'Translator' object has no attribute 'raise_Exception'
Erreur de traduction pour le mot 'management': 'Translator' object has no attribute 'raise_Exception'
Erreur de traduction pour le mot 'operationnel': 'Translator' object has no attribute 'raise_Exception'
Erreur de traduction pour le mot 'achats': 'Translator' object has no attribute 'raise_Exception'
Erreur de traduction pour le mot 'nouvelle': 'Translator' object has no attribute 'raise_Exception'
Erreur de traduction pour le mot 'formule': 'Translator' object has no attribute 'raise_Exception'
Erreur de traduction pour le mot 'formation': 'Translator' object has no attribute 'raise_Exception'
Erreur de traduction pour le mot 'metier': 'Translator' object has no attribute 'raise_Exce

Erreur de traduction pour le mot 'nouvelle': 'Translator' object has no attribute 'raise_Exception'
Erreur de traduction pour le mot 'formule': 'Translator' object has no attribute 'raise_Exception'
Erreur de traduction pour le mot 'economie': 'Translator' object has no attribute 'raise_Exception'
Erreur de traduction pour le mot 'circulaire': 'Translator' object has no attribute 'raise_Exception'
Erreur de traduction pour le mot 'new': 'Translator' object has no attribute 'raise_Exception'
Erreur de traduction pour le mot 'droit': 'Translator' object has no attribute 'raise_Exception'
Erreur de traduction pour le mot 'contrats': 'Translator' object has no attribute 'raise_Exception'
Erreur de traduction pour le mot 'aspects': 'Translator' object has no attribute 'raise_Exception'
Erreur de traduction pour le mot 'juridiques': 'Translator' object has no attribute 'raise_Exception'
Erreur de traduction pour le mot 'achats': 'Translator' object has no attribute 'raise_Exception'
Erreur d

KeyboardInterrupt: 

In [ ]:
import cx_Oracle
from nltk.corpus import wordnet

# Connexion à la base de données
dsn = cx_Oracle.makedsn("localhost", "1521", service_name="orcl")
conn = cx_Oracle.connect(user="sys", password="123", dsn=dsn, mode=cx_Oracle.SYSDBA)

def find_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().replace('_', ' '))
    return list(synonyms)

try:
    cursor = conn.cursor()
    select_query = "SELECT ID, MotTraduit FROM TraductionMots"
    cursor.execute(select_query)
    
    for row in cursor:
        mot_id = row[0]
        mot_traduit = row[1]
        synonymes = find_synonyms(mot_traduit)

        insert_cursor = conn.cursor()
        for synonyme in synonymes:
            insert_query = "INSERT INTO SynonymesMots (ID, MotOriginal, Synonyme) VALUES (:id, :mot_orig, :synonyme)"
            insert_cursor.execute(insert_query, id=mot_id, mot_orig=mot_traduit, synonyme=synonyme)
        conn.commit()
        insert_cursor.close()

finally:
    cursor.close()
    conn.close()
